<a href="https://colab.research.google.com/github/AryanChaks/Llama2-FineTune/blob/main/Orca_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [ ]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Load the dataset from Hugging Face
dataset = load_dataset("Open-Orca/OpenOrca",split="train[:10000]")

In [ ]:
print(dataset)

Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 10000
})


In [ ]:
import torch

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

In [ ]:
import numpy as np

In [ ]:
# Load the pre-trained sentence-transformers model
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# Group instructions using cosine similarity
grouped_instructions = defaultdict(list)

# Iterate over subset dataset and preprocess
for example in dataset:
    response_tokens = tokenizer.tokenize(example["response"])
    if len(response_tokens) >= 100:
        # Encode the response to obtain embeddings
        response_embedding = model.encode(example["response"])

        added_to_group = False

        # Calculate cosine similarity with existing groups
        for group_response_embedding, group_examples in grouped_instructions.items():
            similarity = torch.nn.functional.cosine_similarity(torch.tensor(response_embedding).unsqueeze(0), torch.tensor(group_response_embedding).unsqueeze(0))
            if similarity > 0.95:
                group_examples.append(example)
                added_to_group = True
                break

        # If no group found, create a new one
        if not added_to_group:
            # Convert numpy array to tuple before using as dictionary key
            grouped_instructions[tuple(response_embedding)] = [example]

In [ ]:
for group_examples in grouped_instructions.values():
    print("Group:")
    for example in group_examples:
        print(example)
    print()

In [ ]:
with open('processed_dataset.txt', 'w') as f:
    for group_examples in grouped_instructions.values():
        for example in group_examples:
            f.write(str(example) + '\n')

# Download the file to your local machine
from google.colab import files
files.download('processed_dataset.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('AryanChaks/Grouped_Open_Orca')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


In [ ]:
transformed_dataset.push_to_hub("AryanChaks/transformed_data")